In [9]:
import numpy as np
import cupy as cp
from cupyx import jit

@jit.rawkernel(device=True)
def do_a_thing(x, y):
    return (y, x)
    
@jit.rawkernel()
def do_more_things(x, y, size):
    tid = jit.blockIdx.x * jit.blockDim.x + jit.threadIdx.x
    ntid = jit.gridDim.x * jit.blockDim.x
    for i in range(tid, size, ntid):
        (x[i], y[i]) = do_a_thing(x[i], y[i])

size = 10
x = cp.random.random(size, dtype=np.float32)
y = cp.random.random(size, dtype=np.float32)

print(x)
print(y)

do_more_things((128,),(1024,),(x, y, size))

print(x)
print(y)


[0.73879254 0.36430386 0.5560938  0.13465711 0.49230078 0.8651738
 0.01775549 0.39530468 0.26143515 0.01097945]
[0.96701926 0.31582886 0.45942357 0.02675973 0.44531643 0.38146228
 0.26420838 0.38977566 0.22158593 0.36414465]
[0.96701926 0.31582886 0.45942357 0.02675973 0.44531643 0.38146228
 0.26420838 0.38977566 0.22158593 0.36414465]
[0.73879254 0.36430386 0.5560938  0.13465711 0.49230078 0.8651738
 0.01775549 0.39530468 0.26143515 0.01097945]
